In [10]:
setwd('/home/matt/MSOR/ISYE7406/ML-Ex-Rates/Results')
library(class)
library(MASS)
library(pls)
library(lars)
library(lattice)
library(dplyr)

In [12]:
countries <- c('canada','europe','mexico','japan','korea')
for (country in countries){
    full <- read.csv(paste(country,'_final.csv',sep=''))
    full <- full[,-1]
    rownames(full) <- 1:nrow(full)
    regmodel <- lm(Pct_Chg ~ ., data = full)
    min.model = lm(Pct_Chg ~ 1, data = full)
    max_model <- formula(regmodel)
    linreg_AIC = step(min.model, direction = 'forward',
            scope = max_model, trace = FALSE)
    vars <- as.character(formula(linreg_AIC))
    var_list <- unlist(strsplit(vars[3],' +'))
    var_list <- var_list[c(TRUE,FALSE)]
    var_list <- c('Pct_Chg',var_list)
    aic_vars <- full[var_list]
    print(country)
    print(sort(var_list))
    print('')
}

[1] "canada"
 [1] "BOT_d"            "Exchange"         "FER_f"            "GDPG_d"          
 [5] "GDPG_f"           "Infl_f"           "Int_d"            "Int_f"           
 [9] "lag1.X1Y_Yield_d" "lag1.X1Y_Yield_f" "lag2.Exchange"    "lag2.Infl_d"     
[13] "lag2.Int_f"       "lag2.X1Y_Yield_d" "lag3.Infl_f"      "lag3.Int_d"      
[17] "lag3.X1Y_Yield_d" "lag3.X1Y_Yield_f" "lag4.Exchange"    "lag4.Infl_d"     
[21] "lag4.Infl_f"      "lag4.Int_d"       "lag4.X1Y_Yield_d" "lag4.X1Y_Yield_f"
[25] "Pct_Chg"          "X1Y_Yield_f"     
[1] ""
[1] "europe"
 [1] "BOT_d"            "BOT_f"            "Exchange"         "GDPG_d"          
 [5] "GDPG_f"           "Infl_d"           "Infl_f"           "lag1.X1Y_Yield_f"
 [9] "lag3.Int_d"       "lag4.Infl_f"      "lag4.X1Y_Yield_f" "lag5.Infl_f"     
[13] "lag5.Int_f"       "lag5.X1Y_Yield_d" "lag6.Infl_d"      "lag6.Int_d"      
[17] "lag6.Int_f"       "lag7.X1Y_Yield_d" "lag8.Int_d"       "lag9.Exchange"   
[21] "lag9.Int_d"       "lag9.X1Y

In [8]:
test_final_models <- function(country, B){
         
        full <- read.csv(paste(country,'_final.csv',sep=''))
        full <- full[,-1]
        rownames(full) <- 1:nrow(full)

        #AIC Model
        regmodel <- lm(Pct_Chg ~ ., data = full)
        min.model = lm(Pct_Chg ~ 1, data = full)
        max_model <- formula(regmodel)
        linreg_AIC = step(min.model, direction = 'forward',
                 scope = max_model, trace = FALSE)
        vars <- as.character(formula(linreg_AIC))
        var_list <- unlist(strsplit(vars[3],' +'))
        var_list <- var_list[c(TRUE,FALSE)]
        var_list <- c('Pct_Chg',var_list)
        aic_vars <- full[var_list]

        # Split into training and testing sets
        n <- dim(full)[1]
        n1 <- round(n/5)
        B <- 100
    
        # Test Full Model

        # Linear Regression -- Full Model
        set.seed(19890211)
        linreg.TE_list <- vector(length=0)
        for(i in 1:B){
            start <- sample(1:(n-n1-1),1)
            flag <- (start+6):(start+n1-6)            
            train <- aic_vars[-flag,]; test <- aic_vars[flag,]
    
            regmodel <- lm(Pct_Chg ~ ., data = train)
            linreg.test <- predict.lm(regmodel,test[,-1])
            linreg.TE <- mean( (test[,1]-linreg.test)^2)
            linreg.TE_list <- c(linreg.TE_list, linreg.TE)
    
        }    
    
        # Ridge Regression -- Full Model
        set.seed(19890211)
        ridge.TE_list <- vector(length=0)
        for(i in 1:B){
            start <- sample(1:(n-n1-1),1)
            flag <- (start+6):(start+n1-6)            
            train <- aic_vars[-flag,]; test <- aic_vars[flag,]
    
            ridgereg <- lm.ridge(Pct_Chg ~ ., data = train, 
                        lambda = seq(0,10,0.001))
            lambdaopt <- which.min(ridgereg$GCV)
            ridge_coef <- coef(ridgereg)[lambdaopt,]
            ridge_testmat <- as.matrix(cbind(1,test[,-1]))
            ridge.test <- as.matrix(ridge_testmat %*% ridge_coef)
            ridge.TE <- mean( (test[,1] - ridge.test)^2)
            ridge.TE_list <- c(ridge.TE_list, ridge.TE)
    
        }
    
        # Lasso Regression -- Full Model
        set.seed(19890211)
        lasso.TE_list <- vector(length=0)
        lasso_lambda <- 0
        for(i in 1:B){
            start <- sample(1:(n-n1-1),1)
            flag <- (start+6):(start+n1-6)    
            train <- aic_vars[-flag,]; test <- aic_vars[flag,]
    
            lassoreg <- lars( x= as.matrix(train[,-1]),
                            y = train[,1], type = 'lasso')
            Cp1 <- summary(lassoreg)$Cp
            index1 <- which.min(Cp1)
            lasso_coef <- coef(lassoreg)[index1,]
            if (is.na(lassoreg$lambda[index1])==FALSE){
                lasso_lambda <- lassoreg$lambda[index1]
            }
            lassoreg.fit <- predict(lassoreg, as.matrix(test[,-1]),
                                   s = lasso_lambda, type = 'fit',
                                   mode = 'lambda')
            lassoreg.test <- lassoreg.fit$fit
            lassoreg.TE <- mean( (test[,1]-lassoreg.test)^2)
            lasso.TE_list <- c(lasso.TE_list, lassoreg.TE)
                
        }
    
        # PCR -- Full Model
        set.seed(19890211)
        pcreg.TE_list <- vector(length=0)
        for(i in 1:B){
            start <- sample(1:(n-n1-1),1)
            flag <- (start+6):(start+n1-6)
            train <- aic_vars[-flag,]; test <- aic_vars[flag,]
            
            pcreg <- pcr(Pct_Chg ~ ., data = train,
                        validation = 'CV')
            comps <- dim(train)[2]-1
            pcreg.test <- predict(pcreg, ncomp = comps,
                                 newdata = test[,-1])
            pcreg.TE <- mean((test[,1]-pcreg.test)^2)
            pcreg.TE_list <- c(pcreg.TE_list, pcreg.TE)
        }
    
        # Naive
        set.seed(19890211)
        naive.TE_list <- vector(length=0)
        for(i in 1:B){
            start <- sample(1:(n-n1-1),1)
            flag <- (start+6):(start+n1-6)
            train <- full[-flag,]; test <- full[flag,]
            naive.TE <- mean ( (test[,1])^2)
            naive.TE_list <- c(naive.TE_list,naive.TE)
        }
    
        output1 <- c('Full','Linear','Ridge',
                    'Lasso', 'PCR','Naive')
        output1 <- rbind(output1, c('Mean', mean(linreg.TE_list),
                             mean(ridge.TE_list),mean(lasso.TE_list),
                             mean(pcreg.TE_list), mean(naive.TE_list)))
        output1 <- rbind(output1, c('Variance', var(linreg.TE_list),
                             var(ridge.TE_list), var(lasso.TE_list),
                             var(pcreg.TE_list),var(naive.TE_list)))
        output1 <- rbind(output1, c('t-test',
                t.test(linreg.TE_list,naive.TE_list,alternative='less',paired=TRUE)$p.value,
                t.test(ridge.TE_list,naive.TE_list,alternative='less',paired=TRUE)$p.value,
                t.test(lasso.TE_list,naive.TE_list,alternative='less',paired=TRUE)$p.value,
                t.test(pcreg.TE_list,naive.TE_list,alternative='less',paired=TRUE)$p.value, NA))
        output1 <- rbind(output1,c('t-test2',
                    NA,
                    t.test(ridge.TE_list,linreg.TE_list,paired=TRUE)$p.value,
                    t.test(lasso.TE_list,linreg.TE_list,paired=TRUE)$p.value,
                    t.test(pcreg.TE_list,linreg.TE_list,paired=TRUE)$p.value, NA))

        return(output1)
}

In [9]:
print('canada')
test_final_models('canada',100)
print('europe')
test_final_models('europe',100)
print('mexico')
test_final_models('mexico',100)
print('japan')
test_final_models('japan',100)
print('korea')
test_final_models('korea',100)

[1] "canada"


output1,Full,Linear,Ridge,Lasso,PCR,Naive
,Mean,0.00137447142765265,0.00139509365331705,0.00140981593120451,0.00153293957752987,0.0063689539861169
,Variance,1.40773319104912e-06,1.60081267065167e-06,1.57810489541458e-06,2.19554109645504e-06,4.60193890284921e-05
,t-test,6.43982105020047e-13,4.90227720418683e-13,5.63326297024789e-13,1.81062209344964e-10,NA
,t-test2,NA,0.211702125748568,0.00821351736313249,0.374651498428165,NA


[1] "europe"


output1,Full,Linear,Ridge,Lasso,PCR,Naive
,Mean,0.00415081167615026,0.00438946328170402,0.00434360743543172,0.00386934101265669,0.0115927011512398
,Variance,6.08347272800817e-06,7.39114213111612e-06,6.86050015398912e-06,5.99663516846219e-06,4.57675639981727e-05
,t-test,2.46909486864001e-18,4.33108626281103e-18,3.97996805524047e-18,4.97806521517191e-19,NA
,t-test2,NA,1.76002091648824e-07,3.21732512256471e-07,0.440218701207076,NA


[1] "mexico"


output1,Full,Linear,Ridge,Lasso,PCR,Naive
,Mean,0.00141812914514133,0.00149971538114464,0.00173579819432141,0.00148396227985797,0.0114663924955909
,Variance,1.12618835105025e-06,1.03873530910738e-06,1.64187527032151e-06,9.47272028263573e-07,0.000190995884270143
,t-test,1.61555063302614e-11,1.83319945103005e-11,1.39914028082593e-11,7.63530047369959e-11,NA
,t-test2,NA,0.0001913470757057,3.60237519871321e-07,0.661208187835831,NA


[1] "japan"


output1,Full,Linear,Ridge,Lasso,PCR,Naive
,Mean,0.00984706495614857,0.00984018548984247,0.00962208403305904,0.00960246305993554,0.0205895087693659
,Variance,2.52410629816181e-05,2.45134900061627e-05,2.07012091129938e-05,2.54592879333713e-05,0.000124534464154342
,t-test,8.26534341207276e-15,7.28426398156494e-15,1.3005234716404e-15,2.36915645877713e-14,NA
,t-test2,NA,0.495043656652983,0.00133690095438805,0.725231231021155,NA


[1] "korea"


output1,Full,Linear,Ridge,Lasso,PCR,Naive
,Mean,0.00291995903106997,0.0029316964969929,0.00310904293670996,0.00251949906451078,0.0128872430697519
,Variance,4.19785408942282e-06,4.71137913415253e-06,5.60878953269898e-06,3.30748573111884e-06,0.000175975364851381
,t-test,1.07622059249693e-13,7.53665755083542e-14,7.70415837472509e-14,2.54121097713649e-12,NA
,t-test2,NA,0.547798436305724,7.78682044352837e-06,0.150291996615071,NA
